BERT: http://cedro3.com/ai/bert-pretrained-model/<br>
T5: https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html

In [30]:
# !pip install transformers
# !pip install pytorch_lightning
# !pip install nltk
# !pip install sentencepiece

# T5文章生成

In [3]:
# !git clone https://github.com/dskomei/text_generation.git

/root/myocr/text_generation


In [4]:
# %cd text_generation
# !pip install -r requirements.txt
# !pip install ipywidgets
# !pip install jupyter
# !jupyter nbextension enable --py widgetsnbextension

In [5]:
from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
import settings
import gc
gc.collect()

47

In [6]:
%load_ext autoreload
%autoreload 2
pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)
pd.set_option('max_colwidth', 300)
gc.collect()

0

In [49]:
# !wget -O ldcc-20140209.tar.gz https://www.rondhuit.com/download/ldcc-20140209.tar.gz

In [50]:
# with tarfile.open("ldcc-20140209.tar.gz") as tar:
#     tar.extractall()

In [8]:
file_paths = []
for dir_path in Path('text').glob('*/**'):
    for file_path in dir_path.glob('*'):
        if dir_path.name in file_path.name:
            file_paths.append(file_path)

data = []
for file_path in file_paths:
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]
        title = lines[0]
        body = ''.join(lines[1:])
        data.append((title, body))
data = pd.DataFrame(data, columns=['title', 'body'])
gc.collect()

0

In [7]:
def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text
gc.collect()

0

In [8]:
# data = data.assign(
#     title=lambda x: x.title.map(lambda y: preprocess_text(y)),
#     body=lambda x: x.body.map(lambda y: preprocess_text(y))
# )
# gc.collect()
# data.head()

In [10]:
# df = pd.read_csv('/root/myocr/oran_train_to_T5.csv', index_col=0)
df = pd.read_csv('/root/myocr/oran_train_to_T5_hand_oran.csv', index_col=0)
data = df.rename(columns={'input': 'title', '確認事項': 'body'})

In [11]:
data

,title,body
0,"['U-Plane: User Plane: refers to IQ sample data transferred between O-DU and O-RU', 'o Data Flow 1a: Flows of IQ Data in FFT frequency domain on DL', 'o Data Flow 1b: Flows of IQ Data in FFT frequency domain on UL', 'o Data Flow 1c: Flow of PRACH IQ data in FFT frequency domain', '0000 0000b = I...",The IQ data sent by the simulator shall be received normally.
1,"['6.1.3 Digital Power Scaling', '- Beamforming Support: The same interface design can support different beamforming techniques (digital, analog, hybrid) as well as different beamforming algorithms. Likewise, deployments using only analog beamforming are also possible with the same interface desi...",Make sure you are using Digital Beamforming.
2,"['Low-PHY: those portions of the PHY processing on the O-RU side of the fronthaul interface, including FFT/iFFT, digital beamforming, and PRACH extraction and filtering.', 'O-RU: O-RAN Radio Unit: a logical node hosting Low-PHY layer and RF processing based on a lower layer functional split. Th...",Check the FFT size from the C-Plane of the DL.
3,"['FFO: Fractional Frequency Offset. This is defined as ∆f/f_norm which is used to describe frequency error, typically on the output of the T-TSC filter in the O-RU. It is the same as the Fractional Frequency Deviation defined in ITU-T G.810.', 'T-TSC: Telecom Slave Clock', 'For LTE (e.g. TM9) a...",Check Subcarrier spacing (mu)=1 (30khz) for DL.
4,[],Confirmed that the packet of Payload30Byte is received and becomes on_time.
5,"['o laaMsgType = “0” shall be set for LBT_DL_CONFIG.request: LBT_PDSCH_REQ', 'o laaMsgType = “1” shall be set for LBT_DL_CONFIG.request: LBT_DRS_REQ', 'o laaMsgType = “5” shall be set for LBT_DL_CONFIG.request: LBT_CWCONFIG_REQ']","As shown in Figure 2-13: Uplink and Downlink Timing Parameter Relationship.\nAs shown in Figure 2-13: Uplink and Downlink Timing Parameter Relationship, confirm that the latency does not exceed T12_max."
6,[],"When using Ethernet, confirm that the packets are output as shown in Figure 3-1: Native Ethernet frame with VLAN in the O-RAN(CUS) specification."
7,[],"When using Ethernet, confirm that the packets are output as shown in Figure 3-1: Native Ethernet frame with VLAN in the O-RAN(CUS) specification."
8,"['o laaMsgType = “0” shall be set for LBT_DL_CONFIG.request: LBT_PDSCH_REQ', 'o laaMsgType = “1” shall be set for LBT_DL_CONFIG.request: LBT_DRS_REQ', 'o laaMsgType = “5” shall be set for LBT_DL_CONFIG.request: LBT_CWCONFIG_REQ']",Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).
9,[],Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).


In [18]:
no_data = [i for i in range(len(data)) if data.loc[i, 'title'] == '[]']
print(no_data)

[4, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19, 21, 22, 23, 25, 26, 27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 50, 53, 55, 57, 59, 61, 63, 65, 67, 75, 76, 80, 81, 83, 84, 86, 87, 89, 90, 92, 93, 105]


In [12]:
# data = data.drop(data.index[no_data]).reset_index(drop=True)
# data

In [13]:
def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=settings.max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt

    train_iter = DataLoader(train_data, batch_size=settings.batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=settings.batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter
gc.collect()

0

In [14]:
tokenizer = T5Tokenizer.from_pretrained(settings.MODEL_NAME, is_fast=True)
gc.collect()

4

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    data['title'], data['body'], test_size=0.2, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)
gc.collect()

0

In [16]:
class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(settings.MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )
    
gc.collect()

0

In [17]:
def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(settings.device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(settings.device),
            attention_mask=src['attention_mask'].to(settings.device),
            decoder_attention_mask=tgt['attention_mask'].to(settings.device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
#         pbar.set_description("src: %d " % src)
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        
    return losses / len(data)
gc.collect()

0

In [18]:
def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(settings.device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(settings.device),
                attention_mask=src['attention_mask'].to(settings.device),
                decoder_attention_mask=tgt['attention_mask'].to(settings.device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
        
    return losses / len(data)
gc.collect()

0

In [26]:
# このセル動かなかったやつ↑のコマンド実行してカーネルリスタートしたらできた
model = T5FineTuner()
model = model.to(settings.device)

optimizer = optim.Adam(model.parameters())

PAD_IDX = tokenizer.pad_token_id
best_loss = float('Inf')
best_model = None
counter = 1
        
gc.collect()

279

In [ ]:
# 途中で止まってたやつを再学習させる時用
model_dir_path = Path('hand_oran_model')
tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
model = T5ForConditionalGeneration.from_pretrained(model_dir_path)

In [27]:
# これ時間かかりすぎるのでトレーニングジョブにして実行したい
# これ？https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/step-functions-data-science-sdk/model-train-evaluate-compare/step_functions_mlworkflow_scikit_learn_data_processing_and_model_evaluation_with_experiments.ipynb
for loop in range(1, settings.epochs + 1):

    start_time = time.time()

    loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

    elapsed_time = time.time() - start_time

    loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

    print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
        loop, settings.epochs, loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))

    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = copy.deepcopy(model)
        counter = 1
    else:
        if counter > settings.patience:
            break

        counter += 1
        
    gc.collect()

  0%|          | 0/6 [00:00<?, ?it/s]

[2021-12-06 01:16:17.943 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-2a0e2554e33379262f4fea063cf1:32 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-12-06 01:16:17.976 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-2a0e2554e33379262f4fea063cf1:32 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


  0%|          | 0/6 [00:00<?, ?it/s]

[1/1000] train loss: 4.7771, valid loss: 4.0971 [24s] counter: 1 **


  0%|          | 0/6 [00:00<?, ?it/s]

...

[33/1000] train loss: 0.2052, valid loss: 3.4812 [22s] counter: 19 


  0%|          | 0/6 [00:00<?, ?it/s]

[34/1000] train loss: 0.1752, valid loss: 3.6141 [23s] counter: 20 


100%|██████████| 6/6 [00:22<00:00,  3.72s/it, loss=0.244]


[35/1000] train loss: 0.2437, valid loss: 4.3858 [22s] counter: 21 


# ここから推論

In [19]:
# MODEL_NAME = sonoisa/t5-base-japanese
# model_dir_path = Path('model')
# MODEL_NAME = t5-small
# model_dir_path = Path('g_model')
# model_dir_path = Path('oran_model')
model_dir_path = Path('hand_oran_model')
# model_dir_path = Path('hand_oran_model_1206')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)
gc.collect()

0

In [20]:
tokenizer.save_pretrained(model_dir_path)
model.model.save_pretrained(model_dir_path)
gc.collect()

NameError: name 'model' is not defined

In [21]:
def generate_text_from_model(title, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    title = preprocess_text(title)
    batch = tokenizer(
        [title], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        # num_beams=10,  # ビームサーチの探索幅
        # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=10,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts
gc.collect()

8

In [22]:
def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text
gc.collect()

0

In [23]:
tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    data['title'], data['body'], test_size=0.2, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)
# gc.collect()

In [25]:
# vaild_dataのタイトルを見てみる
valid_title = [valid_data[i][0] for i in range(len(valid_data))]

In [75]:
index = 0
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
【sports watch】プロレスのオールスター戦開催も、猪木は呼ばれず激怒

□ 生成本文
プロレスラー・佐々木則夫&猪木健介の新日本プロレスによるオールスター戦が25日、東京・後楽園ホールで行われた。だが、その試合には特別ルールが存在し、猪木は呼ばれず、永田裕之や蝶野正洋といったスター選手が登場するなど、ファンにとってはその存在意義を感じさせる出来事となった。そんな折、10日(火)発売の「週刊アサヒ芸能」(1/14号)では、『真栄田×アントニオ猪木の闘莉王戦』と題し、自身のブログ上で激しい罵倒を展開している。同誌にコメントを寄せるスポーツ紙デスクは、「今年3月31日に米プロレスフェスタ『ブラック・スワン』を主催している。タイガー・ウッズとのビッグマッチ開催も決まっていたが、この日の午前中に行われたにも関わらず、猪木のファンが入場を拒否したため、猪木は呼ばれず、鳥谷との接触口を開くことはできなかった。それでも、猪木は呼ばれず、蝶野から蝶野が呼ばれず、ちっとも名前はない。しかし、我が家に集うわけにもいかずに、自宅の玄関でチラシを撒き散らす猪木の姿があった。一体なぜ、猪木が呼ばれず、どんな名称で呼ばれるのかー。同誌によると、昨年6月29日(土)午後11時30分から行われる「dynamite!!」というイベントにおいて、猪木が所属するghg対アントニオ猪木の一戦が行われたのだが、当の猪木は呼ばれず、猪木

□ 教師データ本文
先月、東京スポーツ新聞社の主催で、新日本プロレス、全日本プロレス、プロレスリング・ノアの3団体が中心となって行われる東北地方太平洋沖地震の復興支援イベント=「all togeter」の開催が発表された。8月27日(土)日本武道館を舞台に行われる同大会は、79年に同所で行われた夢のオールスター戦を彷彿とさせるもの。当時は、ジャイアント馬場とアントニオ猪木のbi砲がタッグを結成し、アブドーラ・ザ・ブッチャー&タイガー・ジェット・シンの極悪コンビと夢の対戦を行っている。まさに32年ぶりの大会開催と相成るわけだが、プロレス界のアイコン=アントニオ猪木がなんと同イベントからお呼びが掛からず激怒しているというのだ。これは、24日発売の「週刊アサヒ芸能」(6.2号)記事によるもので、同誌は『a猪木が「復興支援大会」に呼ばれず激怒主催東スポと大ゲン

In [76]:
index = 12
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
ありのままの私を愛しなさい!presented byゆるっとcafe

□ 生成本文
はじめまして、まんが家の安斎かなえです。結婚して10年めを目前に、とっても素敵な男性に出逢えて幸せな毎日を送っております。もしかして私のことを好きになってしまったのかも...。ありのままの私を愛してくれている私がいることが本当に幸せなのか疑問です。その秘密に私はあるのです!「ありのままの私を愛しなさい!」byマミさん(32歳・会社員)は、「ありのままの私を愛しなさい!」と言いたい放題で、堅実な出版社勤務のマコさんにご挨拶をさせていただきました。不本意ながらお話しをさせていただくことにさせていただきました。しかし、ありのままのわがままに生きていける自由さにはとても満足できませんでした。でもこのまま生真面目に考えて行動すればいいのに......と、思わず泣き出してしまいました。そんなありのままの私ですが、ありのままの私を愛してくれている皆さんにぜひ聞かせて下さい!!このコラムを読んでくださるみなさまにも、ツッコミを入れたくなるようなこと間違いなしです☆まずは、読み返しながら読んでみてくださいね♪普通に考えれば納得できるはずです。やっぱり恋愛は慎重に行なうべし!異性から愛される女性って案外難しいものなんですねぇ。色々我慢せずに堂々とセックスを楽しむことができるなんて、ただの自尊心が高いだけです。それなのに、すぐhappiness.comの記事を

□ 教師データ本文
「腐女子」という言葉をご存知でしょうか。いわゆる漫画やアニメキャラなどの男性同士の恋愛(ボーイズ・ラブ)を妄想し、愛好する女子のことで、自嘲して「腐った女子」すなわち「腐女子」と称しています。最近はテレビや書籍で紹介されることもあり、耳にされた、もしくは実際友達が腐女子だった!という方もおられるのでは、そして……。腐ってますがソレが何か?どーも、漫画家・うえやま洋介犬の腐った嫁、菜波です。旦那と出会い、結婚して、かれこれ十年。友達みんな一度はこう言います。「趣味を理解してくれる旦那さんでいいなぁ」いやいや。でなきゃ結婚なんて考えられなかった!確かに恥ずかしい趣味であることは認めますが、好きでやっている事であり、特に腐女子とかオタクとかいうものは不治の病のようなものなのでやめることは絶対に無理だと、言い切っておき

In [77]:
index = 13
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
本当に信じられる占いとは?

□ 生成本文
占いに惹かれるものは、その人だけではない。あなたが本当に信じられる占いを見つけられたとき、あなたはどう答えますか?「この占いを信じて人生を迷った」と語るのはミコさん(35歳)。「以前、彼からプロポーズされた時に感じた不安とは違うことがわかりました。それからずっと、『私もそう思ってた!』というメールが届き、すごく嬉しかったんです。でも、実際に鑑定してもらったら『本当に信じられる占いですか?』と一瞬思ったんです。“本当に信じられる占い"といえば、“信じているかどうかの判断基準"とされるのですが、それとは裏腹に、自分が本気で自分を見つめ直したときに考えるべきことはたくさんありました。私は恋愛についてダメだと分かっていても、それは自分自身のせいだと思っていました。だからこそ自分に自信を持つことは難しいけれど、必ず結果が出ると信じて疑わないタイプなので、信頼できる人に相談してみたらいいとアドバイスをいただきました」ミコさんは今年4月から勤務する会社員。仕事を辞めた後、新しい環境で働き方を変えるよう促され、転職を考えていた。しかし、なかなか前向きな話は聞かない。「私の経験上、今まで付き合った人数は5人の友達しかいないし、私が彼のことを本当に知りたいと思っているのかもなんて思っていなかったんですよ。他の占い師に占ってもらったことがあれば、『あのときあの人はこうだったんだ』みたいな安心感がありましたね」(ミ

□ 教師データ本文
新しい季節が始まると、これから先の未来がどう広がっていくのか期待で膨らんでいく。そうなると、ふだん以上に占いが気になったりするものだ。たくさんの占いがあるが、いったいどれが当たるんだろう?ほかの人はどんな占いを信じているんだろう?と、素朴な疑問が湧いてくる。というわけで、さっそく独女のみなさんにアンケートをお願いし、独自に調査をしてみた。「姓名判断が当たると思います!星座とか血液型とか不特定多数の人に当てはまる占いと違って、産まれた日とか名前(漢字)まで一緒という人は少ないと思うから。確率的にも信頼できそう」(茉莉さん/31歳)「私は手相を信じています。手相芸人の島田周平さんがとくにお気に入りで、彼がよく言っている“手相は毎日変わります"という言葉が、すごく信憑性を感じさせるんですよね。私にはky線

In [85]:
# 多分学習データにはないタイトルで遊んでみる
title = '日経平均終値前日より38円高い2万9700円'
# body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
# print('□ 教師データ本文')
# print(body)

□ タイトル
日経平均終値前日より38円高い2万9700円

□ 生成本文
日経平均株価が前日の24日分より38円高い2万9700円となった。日経平均は前週から33円と大きく下落し、終値予想では昨年3月期の連結業績予想を下方修正した格好だ。この背景には市場動向を総合的に勘ぐる必要があり、消費者心理も本質的だったことが裏付けられたようだ。ネット掲示板には「最低賃金で下げるのかよ」「最低賃金上がりすぎだろ」など、日経平均を2ヶ月近く下げた理由に批判的な声が多く寄せられた。一方で、「前の日より価格が下がった割に安かった」「最安値はいつになったら払う」など、日経平均と比べて32円高くなってしまったことを受け、ユーザーからは「公正な競争が激化する」という感想も見られた。【関連情報】・日経平均初値(4万円)=36円■日本経済新聞■関連記事・<日経平均]前日から43円上昇、66%減の4億8400万円・日経平均42.3%第1位通過|生活編>【話題】・日経平均なんと毎月54億円の大赤字【話題】・日経平均すると...今年の夏商戦はどうなる?【話題】・日経平均価格は、現行の相場で約2倍程度下がってることが判明http://news.livedoor.com/article/detail/5950395/・11月号記事連動「音の編集講座



In [103]:
# 英語タイトルにしたらどうなる？
# 英語タイトル記事：https://www.bloomberg.com/news/articles/2021-11-16/biden-xi-begin-summit-aiming-to-ease-tensions-in-relationship?srnd=premium-asia

title = 'Biden, Xi Seek Cooperation in Longer-Than-Expected Summit'
# body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()

□ タイトル
Biden, Xi Seek Cooperation in Longer-Than-Expected Summit

□ 生成本文
「xi seek cooperation in longer than expected summit」は、昨年3月に東京・青山にオープンした、オープン記念イベント「biden×xi seek cooperation in longer than expected summit」(以下、case1)。同イベントでは、case1の展示会場において、30名程度の一般来場者と300名が招待され、中高生400名を招待した。この模様は、twitterやfacebookを介しても閲覧可能で、id/パスワードの問題がない点が大きな特長だ。また、同時に行われたcooperationについても、その一部を紹介しよう。○xi seek cooperation in longer than expected summit概要●xi seek cooperation in longer than expected summit日時:2012年3月23日(水)10:0025:00場所:メイン会場:ginza3f(東京都港区六本木7-14-1)住所:東京都中央区丸の内3-5-12三越谷5-7-6営業時間:9:0017:00定休日:年末年始【問い合わせ先】お客様ご相談センターフリーダイヤル0120-865受付時間12:0020:00※混雑状況により、受付終了時間が早まる場合があります。詳しくは、こちらをご覧ください。■xi seek cooperation in longer than expected summit日程:2012年3月



まぁそりゃダメですよね

In [110]:
# 単純な事前学習済みt5-smallモデルなら？

title = 'New Zealand’s Public Sector Gender Pay Gap Narrows to Record Low'
# body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()

□ タイトル
New Zealand’s Public Sector Gender Pay Gap Narrows to Record Low

□ 生成本文
Die mangelnde Bezahlungsquote in den private sector geschlechterspezifischem Geschlechterverhältnis im neuen Zzealand liegt bei einem record low, der sich bis zu einer record-breaking niedrigen.



ダメダメだけどとりあえず英語っぽくはなる

In [20]:
# oran仕様書文章から試みる(ocr ver)

valid_data_index = list()
for i in range(len(valid_data)):
    if valid_data[i][0] == '[]':
        pass
    else:
        valid_data_index.append(i)
        
for i in valid_data_index:
    index = i
    title = valid_data[index][0]
    body = valid_data[index][1]
    generated_texts = generate_text_from_model(
        title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
    )
    print('□ タイトル')
    print(title[-10:])
    print()
    print('□ 生成本文')
    print(generated_texts[0])
    print()
    print('□ 教師データ本文')
    print(body)
    print('*'*100)

[2021-11-29 08:47:52.523 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-2a0e2554e33379262f4fea063cf1:31 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-29 08:47:52.557 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-2a0e2554e33379262f4fea063cf1:31 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
□ タイトル
+29/31* ']

□ 生成本文
Verify that the value of ecpriPayload is output correctly.

□ 教師データ本文
U-Plane must be output as shown in "Table 6-2: IQ data frame format".
□ タイトル
 2 bits ']

□ 生成本文
If the value of ecpriConcatenation is low, Rx_late may be applied to the Window of the received data._x000D_ Make sure that when it's 1.

□ 教師データ本文
Verify that the value of StartSymbol is output successfully.
□ タイトル
atency) ']

□ 生成本文
Ensure that the transition to FREERUNState is made when Announce messages are received in PTP.

□ 教師データ本文
1024 (1010[bin]) must be output.
□ タイトル
nalysis ']

□ 生成本文
Ensure that the transition to FREERUNState is

In [115]:
# oran仕様書文章から試みる(hand-oran ver)

valid_data_index = list()
yes_data_index = list()
no_data_index = list()
result = list()
for i in range(len(valid_data)):
    if valid_data[i][0] == '[]':
        no_data_index.append(i)
        valid_data_index.append(i)
    else:
        valid_data_index.append(i)
        yes_data_index.append(i)
#     valid_data_index.append(i)
        
for i in valid_data_index:
    index = i
    title = valid_data[index][0]
    body = valid_data[index][1]
    generated_texts = generate_text_from_model(
        title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
    )
    result.append(generated_texts[0])
    print('□ タイトル')
    print(title[-10:])
    print()
    print('□ 生成本文')
    print(generated_texts[0])
    print()
    print('□ 教師データ本文')
    print(body)
    print('*'*100)

□ タイトル
annels):']

□ 生成本文
5.4.2 Check the list of fields of SectionType3 described in Scheduling and Beamforming Command.

□ 教師データ本文
U-Plane must be output as shown in "Table 6-2: IQ data frame format".
****************************************************************************************************
□ タイトル
[]

□ 生成本文
Check the value of CC_ID.

□ 教師データ本文
Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).
****************************************************************************************************
□ タイトル
[]

□ 生成本文
Check the value of CC_ID.

□ 教師データ本文
Confirmed that the packet of Payload30Byte is received and becomes on_time.
****************************************************************************************************
□ タイトル
[]

□ 生成本文
Check the value of CC_ID.

□ 教師データ本文
If the data is received too early, Rx_early is applied to the Window of the received data._x000D_
Make sure that the relevant parameter is counted up.
*************************************

In [33]:
# oran仕様書文章から試みる、余計なデータは削除ver(hand-oran ver)
# train lossはいいが、val lossがひどい
# 学習データが少ないから？

valid_data_index = list()
for i in range(len(valid_data)):
    if valid_data[i][0] == '[]':
        pass
    else:
        valid_data_index.append(i)
        
for i in valid_data_index:
    index = i
    title = valid_data[index][0]
    body = valid_data[index][1]
    generated_texts = generate_text_from_model(
        title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
    )
    print('□ タイトル')
    print(title[-10:])
    print()
    print('□ 生成本文')
    print(generated_texts[0])
    print()
    print('□ 教師データ本文')
    print(body)
    print('*'*100)

□ タイトル
NFIG_REQ']

□ 生成本文
If the received data is corrupted, Rx_corrpt is applied._x000D_ Make sure that lvcms_make sure it’ll be counten up to date when possible in the case of C-PlanE_Additional Parameter (1).

□ 教師データ本文
Verify that the SubframeID value is output correctly.
****************************************************************************************************
□ タイトル
 length.']

□ 生成本文
When the M-PlaneState transitions to Disconnected, the rfp is applied.

□ 教師データ本文
When the M-PlaneState transitions to Disconnected, it shall be confirmed that the RF output is not turned Off.
****************************************************************************************************
□ タイトル
urable):']

□ 生成本文
If the data is received too early, o-ru |te_corrpt is applied.

□ 教師データ本文
If the Quality Level and Clock Class are acceptable when the PTP Announce message is received, confirm to continue using the reference signal.
***********************************************************

In [29]:
# !pip install openpyxl
# %cd ..

In [47]:
import pandas as pd

df_y = pd.DataFrame(y_test)
df_y

,body
78,"U-Plane must be output as shown in ""Table 6-2: IQ data frame format""."
10,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).
4,Confirmed that the packet of Payload30Byte is received and becomes on_time.
84,"If the data is received too early, Rx_early is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up."
64,Verify that the value of StartSymbol is output successfully.
68,1024 (1010[bin]) must be output.
30,Check the value of BandSector_ID.
45,Check the value of FrequencyOffset.
96,"When the M-PlaneState transitions to Disconnected, the RF output shall be confirmed to be Off."
11,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).


In [117]:
df_y = pd.DataFrame(y_test)
valid_index = list(df_y.index)

In [65]:
import pandas as pd
import numpy as np

sample = pd.read_excel('oran_test_content.xlsx', engine='openpyxl', header=1).reset_index(drop=True)\
                        .drop(['Unnamed: 0', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 14'], axis=1)\
                        .drop(['大項目', '中項目', '小項目1', '小項目2', '小項目3', '小項目4'], axis=1)\
                        .dropna(how='any').reset_index(drop=True)
# sample

,Reference,項番,確認事項
0,2.1.0.0.0.1,1.1.1,シミュレータが送信したIQデータを正常に受信すること。
1,2.1.0.0.0.3,1.1.2,Digital Beamformingを使用していることを確認する。
2,2.2.0.0.2.1,1.1.3,DLのC-PlaneからFFT sizeを確認する。
3,2.2.0.0.2.2,1.1.4,DLのSubcarrier spacing (mu)=1(30khz)を確認する。
4,2.3.3.1,1.1.5.1,Payload30Byteのパケットを受信し、on_timeとなることを確認。\n\n下記の確認を行う。\nT12_minij : T12_min between DU_Port_ID(i) and RU_Port_ID(j)
5,2.3.4,1.1.6.1,「Figure 2-13 : Uplink and Downlink Timing Parameter Relationship」の図の通り、\nLatencyがT12_max以上とならないことを確認する。
6,3.1.1,1.2.1.1,Ethernetを使用した場合、O-RAN(CUS)仕様書のFigure 3-1 : Native Ethernet frame with VLAN通りのパケットが出力されることを確認する。
7,3.1.1,1.2.1.2,Ethernetを使用した場合、O-RAN(CUS)仕様書のFigure 3-1 : Native Ethernet frame with VLAN通りのパケットが出力されることを確認する。
8,3.1.3.1.1,1.2.2.1,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。
9,3.1.3.1.1,1.2.2.2,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。


In [31]:
%%writefile upload_file.py
# ファイルをフォルダごとアップロードするスクリプト

import boto3
import os
import sys

client = boto3.client('s3')
bucket = 'sagemaker-us-east-2-523358537305'

def upload_files(basedir):
    parent_dir = os.path.dirname(os.path.realpath(basedir))
    for (path, dirs, files) in os.walk(basedir):
        for fn in files:
            if fn.startswith('.'):
                continue
            abspath = os.path.join(path, fn)
            yield (
                abspath,
                os.path.relpath(abspath, parent_dir).split(''.join([basedir, '/']))[1]
            )
            
dirname = 'spec2test'
pref = 'test_items/data'
for abspath, relpath in upload_files(dirname):
    client.upload_file(abspath, bucket, pref+abspath)
#     print(abspath)
#     print(relpath)

Filename = '/root/myocr/hand-oran.txt'
Key = 'hand-oran.txt'
pref = 'test_items/data/'
client.upload_file(Filename, bucket, pref+Key)

Writing upload_file.py


In [32]:
dirname = 'spec2test'
pref = 'test_items/data'
for abspath, relpath in upload_files(dirname):
    client.upload_file(abspath, bucket, pref+abspath)
#     print(abspath)
#     print(relpath)

In [33]:
Filename = '/root/myocr/hand-oran.txt'
Key = 'hand-oran.txt'
pref = 'test_items/data/'
client.upload_file(Filename, bucket, pref+Key)

In [67]:
valid_grandtruth = sample.iloc[valid_index]
# valid_grandtruth

,Reference,項番,確認事項
78,6.3.2.0.0.2,1.4.4,「Table 6-2 : IQ data frame format」の通り、U-Planeが出力されること。
10,3.1.3.1.1,1.2.2.3,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。
4,2.3.3.1,1.1.5.1,Payload30Byteのパケットを受信し、on_timeとなることを確認。\n\n下記の確認を行う。\nT12_minij : T12_min between DU_Port_ID(i) and RU_Port_ID(j)
84,7.1.0.0.0.2,1.5.2.3,データの受信が早すぎる場合、Rx_earlyを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する
64,5.4.4.7,1.3.12.1,StartSymbolの値が正常に出力されていることを確認する。
68,5.4.4.13.0.1,1.3.14,1024(1010[bin])が出力されていること。
30,3.1.3.1.6.3,1.2.7.3,BandSector_IDの値を確認する。
45,5.3.2.1,1.3.2.1,FrequencyOffsetの値を確認する。
96,9.4.2.1.0.2,1.6.3,M-PlaneStateがDisconnectedに遷移した際、RF出力がOffになることを確認する。
11,3.1.3.1.1,1.2.2.4,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。


In [68]:
tmp = pd.concat([valid_grandtruth, df_y], axis=1).rename(columns={'body': '確認事項_英語'})
tmp

,Reference,項番,確認事項,確認事項_英語
78,6.3.2.0.0.2,1.4.4,「Table 6-2 : IQ data frame format」の通り、U-Planeが出力されること。,"U-Plane must be output as shown in ""Table 6-2: IQ data frame format""."
10,3.1.3.1.1,1.2.2.3,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).
4,2.3.3.1,1.1.5.1,Payload30Byteのパケットを受信し、on_timeとなることを確認。\n\n下記の確認を行う。\nT12_minij : T12_min between DU_Port_ID(i) and RU_Port_ID(j),Confirmed that the packet of Payload30Byte is received and becomes on_time.
84,7.1.0.0.0.2,1.5.2.3,データの受信が早すぎる場合、Rx_earlyを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received too early, Rx_early is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up."
64,5.4.4.7,1.3.12.1,StartSymbolの値が正常に出力されていることを確認する。,Verify that the value of StartSymbol is output successfully.
68,5.4.4.13.0.1,1.3.14,1024(1010[bin])が出力されていること。,1024 (1010[bin]) must be output.
30,3.1.3.1.6.3,1.2.7.3,BandSector_IDの値を確認する。,Check the value of BandSector_ID.
45,5.3.2.1,1.3.2.1,FrequencyOffsetの値を確認する。,Check the value of FrequencyOffset.
96,9.4.2.1.0.2,1.6.3,M-PlaneStateがDisconnectedに遷移した際、RF出力がOffになることを確認する。,"When the M-PlaneState transitions to Disconnected, the RF output shall be confirmed to be Off."
11,3.1.3.1.1,1.2.2.4,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).


In [79]:
df_result = pd.DataFrame(result, index=valid_index).rename(columns={0: '推論結果_英語'})
df_result

,推論結果_英語
78,5.4.2 Check the list of fields of SectionType3 described in Scheduling and Beamforming Command.
10,Check the value of CC_ID.
4,Check the value of CC_ID.
84,Check the value of CC_ID.
64,Check the value of CC_ID.
68,Check the ecpriPayload.
30,Check the value of CC_ID.
45,Check the value of CC_ID.
96,"When the M-PlaneState transitions to Disconnected, it shall be confirmed that its rf output is not turned Off."
11,Check the value of CC_ID.


In [84]:
df_all = pd.concat([tmp, df_result], axis=1).drop(['Reference', '項番'], axis=1)
df_all['推論結果_日本語'] = '[]'

In [85]:
df_all

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語
78,「Table 6-2 : IQ data frame format」の通り、U-Planeが出力されること。,"U-Plane must be output as shown in ""Table 6-2: IQ data frame format"".",5.4.2 Check the list of fields of SectionType3 described in Scheduling and Beamforming Command.,[]
10,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,Check the value of CC_ID.,[]
4,Payload30Byteのパケットを受信し、on_timeとなることを確認。\n\n下記の確認を行う。\nT12_minij : T12_min between DU_Port_ID(i) and RU_Port_ID(j),Confirmed that the packet of Payload30Byte is received and becomes on_time.,Check the value of CC_ID.,[]
84,データの受信が早すぎる場合、Rx_earlyを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received too early, Rx_early is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up.",Check the value of CC_ID.,[]
64,StartSymbolの値が正常に出力されていることを確認する。,Verify that the value of StartSymbol is output successfully.,Check the value of CC_ID.,[]
68,1024(1010[bin])が出力されていること。,1024 (1010[bin]) must be output.,Check the ecpriPayload.,[]
30,BandSector_IDの値を確認する。,Check the value of BandSector_ID.,Check the value of CC_ID.,[]
45,FrequencyOffsetの値を確認する。,Check the value of FrequencyOffset.,Check the value of CC_ID.,[]
96,M-PlaneStateがDisconnectedに遷移した際、RF出力がOffになることを確認する。,"When the M-PlaneState transitions to Disconnected, the RF output shall be confirmed to be Off.","When the M-PlaneState transitions to Disconnected, it shall be confirmed that its rf output is not turned Off.",[]
11,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,Check the value of CC_ID.,[]


In [106]:
# 推論結果を翻訳してdfに格納
# AWS Translateにて
# import boto3

# translate = boto3.client('translate')
index = list(df_all.index)

for ind in index:
    result = translate.translate_text(Text=df_all.loc[ind, '推論結果_英語'], SourceLanguageCode='en', TargetLanguageCode='ja')
    df_all.loc[ind, '推論結果_日本語'] = result['TranslatedText']

In [107]:
df_all

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語
78,「Table 6-2 : IQ data frame format」の通り、U-Planeが出力されること。,"U-Plane must be output as shown in ""Table 6-2: IQ data frame format"".",5.4.2 Check the list of fields of SectionType3 described in Scheduling and Beamforming Command.,5.4.2 スケジューリングとビームフォーミングコマンドで説明されている SectionType3 のフィールドの一覧を確認する。
10,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,Check the value of CC_ID.,CC_ID の値を確認してください。
4,Payload30Byteのパケットを受信し、on_timeとなることを確認。\n\n下記の確認を行う。\nT12_minij : T12_min between DU_Port_ID(i) and RU_Port_ID(j),Confirmed that the packet of Payload30Byte is received and becomes on_time.,Check the value of CC_ID.,CC_ID の値を確認してください。
84,データの受信が早すぎる場合、Rx_earlyを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received too early, Rx_early is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up.",Check the value of CC_ID.,CC_ID の値を確認してください。
64,StartSymbolの値が正常に出力されていることを確認する。,Verify that the value of StartSymbol is output successfully.,Check the value of CC_ID.,CC_ID の値を確認してください。
68,1024(1010[bin])が出力されていること。,1024 (1010[bin]) must be output.,Check the ecpriPayload.,ecpriPayload を確認します。
30,BandSector_IDの値を確認する。,Check the value of BandSector_ID.,Check the value of CC_ID.,CC_ID の値を確認してください。
45,FrequencyOffsetの値を確認する。,Check the value of FrequencyOffset.,Check the value of CC_ID.,CC_ID の値を確認してください。
96,M-PlaneStateがDisconnectedに遷移した際、RF出力がOffになることを確認する。,"When the M-PlaneState transitions to Disconnected, the RF output shall be confirmed to be Off.","When the M-PlaneState transitions to Disconnected, it shall be confirmed that its rf output is not turned Off.",M-PlaneState が Disconnected に移行すると、その rf 出力がオフになっていないことが確認されます。
11,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,Check the value of CC_ID.,CC_ID の値を確認してください。


In [114]:
# 仕様書本文内からヒントの単語を基に文章を検索できなかった推論結果
df_all.iloc[no_data_index, :]

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語
10,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,Check the value of CC_ID.,CC_ID の値を確認してください。
4,Payload30Byteのパケットを受信し、on_timeとなることを確認。\n\n下記の確認を行う。\nT12_minij : T12_min between DU_Port_ID(i) and RU_Port_ID(j),Confirmed that the packet of Payload30Byte is received and becomes on_time.,Check the value of CC_ID.,CC_ID の値を確認してください。
84,データの受信が早すぎる場合、Rx_earlyを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received too early, Rx_early is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up.",Check the value of CC_ID.,CC_ID の値を確認してください。
30,BandSector_IDの値を確認する。,Check the value of BandSector_ID.,Check the value of CC_ID.,CC_ID の値を確認してください。
45,FrequencyOffsetの値を確認する。,Check the value of FrequencyOffset.,Check the value of CC_ID.,CC_ID の値を確認してください。
11,ecpriVersionの値が1(eCPRI version 1.0)であることを確認する。,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,Check the value of CC_ID.,CC_ID の値を確認してください。
80,データを時間通りに受信できた場合、Rx_on_timeを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received on time, Rx_on_time is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up.",Check the value of CC_ID.,CC_ID の値を確認してください。
81,データを時間通りに受信できた場合、Rx_on_timeを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received on time, Rx_on_time is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up.",Check the value of CC_ID.,CC_ID の値を確認してください。
18,C-Planeの場合、0x2が出力されていることを確認する。,"In the case of C-Plane, confirm that 0x2 is output.",Check the value of CC_ID.,CC_ID の値を確認してください。
36,Sequence IDの値がインクリメントされることを確認する。,Make sure that the value of Sequence ID is incremented.,Check the value of CC_ID.,CC_ID の値を確認してください。


In [116]:
# 仕様書本文内からヒントの単語を基に文章を検索できた推論結果
df_all.iloc[yes_data_index, :]

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語
78,「Table 6-2 : IQ data frame format」の通り、U-Planeが出力されること。,"U-Plane must be output as shown in ""Table 6-2: IQ data frame format"".",5.4.2 Check the list of fields of SectionType3 described in Scheduling and Beamforming Command.,5.4.2 スケジューリングとビームフォーミングコマンドで説明されている SectionType3 のフィールドの一覧を確認する。
64,StartSymbolの値が正常に出力されていることを確認する。,Verify that the value of StartSymbol is output successfully.,Check the value of CC_ID.,CC_ID の値を確認してください。
68,1024(1010[bin])が出力されていること。,1024 (1010[bin]) must be output.,Check the ecpriPayload.,ecpriPayload を確認します。
96,M-PlaneStateがDisconnectedに遷移した際、RF出力がOffになることを確認する。,"When the M-PlaneState transitions to Disconnected, the RF output shall be confirmed to be Off.","When the M-PlaneState transitions to Disconnected, it shall be confirmed that its rf output is not turned Off.",M-PlaneState が Disconnected に移行すると、その rf 出力がオフになっていないことが確認されます。
79,データを時間通りに受信できた場合、Rx_on_timeを受信したデータのWindowに適用する。\n該当パラメータがカウントアップしていることを確認する,"If the data is received on time, Rx_on_time is applied to the Window of the received data._x000D_\nMake sure that the relevant parameter is counted up.",Check the value of CC_ID.,CC_ID の値を確認してください。
0,シミュレータが送信したIQデータを正常に受信すること。,The IQ data sent by the simulator shall be received normally.,"When the M-PlaneState transitions to Disconnected, it shall be confirmed that the U.264 is output correctly in ""Table 5-1 : IQ data frame format"".",M-PlaneState が Disconnected に移行すると、U.264 が「表 5-1: IQ データフレーム形式」で正しく出力されていることを確認する必要があります。
70,ExtType=1が出力されることを確認する。,Verify that ExtType=1 is output.,Check bfwcomphdr.,bfwcomphdr をチェックしてください。
56,0であることを確認する。,Check that it is 0.,Check the value of CC_ID.,CC_ID の値を確認してください。
72,bfwI_0~31まで出力されていることを確認する。,Confirm that the output is from bfwI_0~31.,Confirm that the output is from bfwi_031.,bfwi_031 からの出力であることを確認します。
108,GPS secondsをPTPから受信したとき特に問題が発生しないことを確認。,Confirmed that no particular problem occurs when GPS seconds are received from PTP.,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).,ecpriVersion の値が 1 (eCPRI バージョン 1.0) であることを確認します。


In [98]:
import boto3
translate = boto3.client('translate')

In [103]:
result = translate.translate_text(Text=df_all.loc[0, '推論結果_英語'], SourceLanguageCode='en', TargetLanguageCode='ja')

In [104]:
result

{'TranslatedText': 'M-PlaneState が Disconnected に移行すると、U.264 が「表 5-1: IQ データフレーム形式」で正しく出力されていることを確認する必要があります。',
 'SourceLanguageCode': 'en',
 'TargetLanguageCode': 'ja',
 'ResponseMetadata': {'RequestId': 'c08c3faa-4dcb-403d-a7e2-966bf0d14b99',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c08c3faa-4dcb-403d-a7e2-966bf0d14b99',
   'cache-control': 'no-cache',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '256',
   'date': 'Tue, 07 Dec 2021 07:51:10 GMT'},
  'RetryAttempts': 0}}

In [101]:
result = translate.translate_text(Text=df_all.loc[0, '確認事項_英語'], SourceLanguageCode='en', TargetLanguageCode='ja')
result

{'TranslatedText': 'シミュレータから送信されたIQデータは正常に受信されます。',
 'SourceLanguageCode': 'en',
 'TargetLanguageCode': 'ja',
 'ResponseMetadata': {'RequestId': '905fd393-a71e-4be0-b764-bf5ee06fcd68',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '905fd393-a71e-4be0-b764-bf5ee06fcd68',
   'cache-control': 'no-cache',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '156',
   'date': 'Tue, 07 Dec 2021 07:46:28 GMT'},
  'RetryAttempts': 0}}

# 水位計＋ORAN

https://huggingface.co/google/t5-small-ssm-nq<br>
### wandb loging 5ce3638828b2e9a963129f637e0ab4e22a2fe677

1/19メモ：max_length_srcが短すぎる？→長さを変えてやってみる<br>


In [15]:
import pandas as pd
import boto3
from io import StringIO

bucket = 'sagemaker-us-east-2-523358537305'
pref = 'test_items/data/'
file_name = 'oran_train_to_T5_hand_oran_1_7.csv'
data_pref = f's3://{bucket}/{pref}/{file_name}'

s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key=pref+file_name)
body = obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string), index_col=0).rename(columns={'input': 'title', '確認事項': 'body'})
df

,title,body
0,3.1.3.1.6 ecpriRtcid / ecpriPcid (real time co...,The IQ data sent by the simulator shall be rec...
1,6.1.3 Digital Power Scaling 96 6.1.3 Digital P...,Make sure you are using Digital Beamforming.
2,Low-PHY: those portions of the PHY processing ...,Check the FFT size from the C-Plane of the DL.
3,5.3.4.3.1 PDSCH Transmission 55 5.4.5.22 lbtPd...,Check Subcarrier spacing (mu)=1 (30khz) for DL.
4,2.1.1 Selected Split 7-2x (DL) 14 5.3.3 DL Pre...,Confirmed that the packet of Payload30Byte is ...
...,...,...
105,NaN,Confirm that domainNumber is set to 24 (FTS IT...
106,9.2.4.2 PTP 120 9.3.1 Allowed PTP and SyncE cl...,Make sure Clock Class is set to 6 or 7 in Anno...
107,9.2.4.2 PTP 120 9.3.1 Allowed PTP and SyncE cl...,Verify that the reference time is January 1970...
108,9.7.2 System Frame Number Calculation from GPS...,Confirmed that no particular problem occurs wh...


In [4]:
# 使うモジュール、関数を最初に定義
# train.pyの関数定義部

from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Model
import settings
import gc
import numpy as np
import boto3
from transformers import pipeline
from io import StringIO

BUCKET = 'sagemaker-us-east-2-523358537305'
pref = 'test_items/data/'
FILE_NAME = 'oran_train_to_T5_hand_oran_1_7.csv'
out_model_dir = 

# 使用するデータ（csvファイル）を読み込む関数
def read():
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=pref+FILE_NAME)
    body = obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string), index_col=0).rename(columns={'input': 'title', '確認事項': 'body'})
    no_data = [i for i in range(len(df)) if df.loc[i, 'title'] is np.nan]
    new_df = df.drop(df.index[no_data]).reset_index(drop=True)
    return new_df

# 読み込んだdataframeをtrain_iterとvalid_iterに分割する関数
def valid_train_test(df, tokenizer):
    random_state = 94
    X_train, X_test, y_train, y_test = train_test_split(
        df['title'], df['body'], test_size=0.2, random_state=random_state, shuffle=True
    )

    train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
    valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

    train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)
    return train_iter, valid_iter

def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=settings.max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt
    
    train_iter = DataLoader(train_data, batch_size=settings.batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=settings.batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(settings.MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(settings.device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(settings.device),
            attention_mask=src['attention_mask'].to(settings.device),
            decoder_attention_mask=tgt['attention_mask'].to(settings.device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
#         pbar.set_description("src: %d " % src)
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        wandb.log({"train_loss": losses / loop})
    return losses / len(data)

def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(settings.device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(settings.device),
                attention_mask=src['attention_mask'].to(settings.device),
                decoder_attention_mask=tgt['attention_mask'].to(settings.device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
            
            wandb.log({"valid_loss": losses})
    return losses / len(data)

def generate_text_from_model(title, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    title = preprocess_text(title)
    batch = tokenizer(
        [title], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        num_beams=25,  # ビームサーチの探索幅
        diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        num_beam_groups=25,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

# BARTの事前学習済みモデルを使って抜き出した文章を指定単語長に要約する
def txt_summarization_by_bart(df):
    smr_bart = pipeline(task=settings.task_name, model=settings.sum_model_from_bart)
    records = len(df)
    for record in range(records):
        src_text = df.loc[record, 'title']
        smbart = smr_bart(src_text, max_length=settings.max_length_src, truncation=True)
        df.loc[record, 'title'] = smbart[0]["summary_text"]
    
    return df

# コサイン類似度計算
def vectorize(text: str):
#     encoding = tokenizer(text, padding = 'longest', return_tensors='pt')
#     outputs = model(**encoding)
    tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
    model = T5Model.from_pretrained(model_dir_path)
    input_ids = tokenizer(text, return_tensors="pt").input_ids  # Batch size 1
    decoder_input_ids = tokenizer(text, return_tensors="pt").input_ids  # Batch size 1
    outputs = model(input_ids, decoder_input_ids=decoder_input_ids)

    return outputs.last_hidden_state[0][0]

def cos_sim(v1, v2):
    v1 = v1.detach().numpy()
    v2 = v2.detach().numpy()
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


# %load_ext autoreload
# %autoreload 2
# pd.set_option('max_rows', 1000)
# pd.set_option('max_columns', 1000)
# pd.set_option('max_colwidth', 300)

# gc.collect()

7

In [1]:
%%writefile train.py
# 使うモジュール、関数を最初に定義
# train.pyの関数定義部
from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Model
import settings
import gc
import numpy as np
import boto3
from transformers import pipeline
from io import StringIO

BUCKET = 'sagemaker-us-east-2-523358537305'
pref = 'test_items/data/'
FILE_NAME = 'oran_train_to_T5_hand_oran_1_7.csv'

# 使用するデータ（csvファイル）を読み込む関数
def read():
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=pref+FILE_NAME)
    body = obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string), index_col=0).rename(columns={'input': 'title', '確認事項': 'body'})
    no_data = [i for i in range(len(df)) if df.loc[i, 'title'] is np.nan]
    new_df = df.drop(df.index[no_data]).reset_index(drop=True)
    return new_df

# 読み込んだdataframeをtrain_iterとvalid_iterに分割する関数
def valid_train_test(df, tokenizer):
    random_state = 94
    X_train, X_test, y_train, y_test = train_test_split(
        df['title'], df['body'], test_size=0.2, random_state=random_state, shuffle=True
    )

    train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
    valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

    train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)
    return train_iter, valid_iter

def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=settings.max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt
    
    train_iter = DataLoader(train_data, batch_size=settings.batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=settings.batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(settings.MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(settings.device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(settings.device),
            attention_mask=src['attention_mask'].to(settings.device),
            decoder_attention_mask=tgt['attention_mask'].to(settings.device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
#         pbar.set_description("src: %d " % src)
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        wandb.log({"train_loss": losses / loop})
    return losses / len(data)

def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(settings.device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(settings.device),
                attention_mask=src['attention_mask'].to(settings.device),
                decoder_attention_mask=tgt['attention_mask'].to(settings.device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
            
            wandb.log({"valid_loss": losses})
    return losses / len(data)

def generate_text_from_model(title, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    title = preprocess_text(title)
    batch = tokenizer(
        [title], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        num_beams=25,  # ビームサーチの探索幅
        diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        num_beam_groups=25,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

# BARTの事前学習済みモデルを使って抜き出した文章を指定単語長に要約する
def txt_summarization_by_bart(df):
    smr_bart = pipeline(task=settings.task_name, model=settings.sum_model_from_bart)
    records = len(df)
    for record in range(records):
        src_text = df.loc[record, 'title']
        smbart = smr_bart(src_text, max_length=settings.max_length_src, truncation=True)
        df.loc[record, 'title'] = smbart[0]["summary_text"]
    
    return df

# コサイン類似度計算
def vectorize(text: str):
#     encoding = tokenizer(text, padding = 'longest', return_tensors='pt')
#     outputs = model(**encoding)
    tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
    model = T5Model.from_pretrained(model_dir_path)
    input_ids = tokenizer(text, return_tensors="pt").input_ids  # Batch size 1
    decoder_input_ids = tokenizer(text, return_tensors="pt").input_ids  # Batch size 1
    outputs = model(input_ids, decoder_input_ids=decoder_input_ids)

    return outputs.last_hidden_state[0][0]

def cos_sim(v1, v2):
    v1 = v1.detach().numpy()
    v2 = v2.detach().numpy()
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


# train.pyのmain部
if __name__ == "__main__":

    # load datasets
    df = read()
    # BARTで、抜き出してきた文章の要約を行う関数（T5で文章生成による学習を行う際に、重要と思われる情報に短くまとめる為）
    new_df = txt_summarization_by_bart(df)
    # T5Tokenizer定義
    tokenizer = T5Tokenizer.from_pretrained(settings.MODEL_NAME, is_fast=True)
    # train_iter, valid_iterに分割
    train_iter, valid_iter = valid_train_test(df, tokenizer)
    
    model = T5FineTuner()
    model = model.to(settings.device)

    optimizer = optim.Adam(model.parameters())

    PAD_IDX = tokenizer.pad_token_id
    best_loss = float('Inf')
    best_model = None
    counter = 1
    gc.collect()
    
    for loop in range(1, settings.epochs + 1):
        start_time = time.time()
        loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)
        elapsed_time = time.time() - start_time
        loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

        print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
            loop, settings.epochs, loss_train, loss_valid,
            str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
            elapsed_time % 60,
            counter,
            '**' if best_loss > loss_valid else ''
        ))

        if best_loss > loss_valid:
            best_loss = loss_valid
            best_model = copy.deepcopy(model)
            counter = 1
        else:
            if counter > settings.patience:
                break
            counter += 1
        gc.collect()

Writing train.py


In [3]:
%%writefile predictor.py
# predictor.py
def generate_text_from_model(title, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    title = preprocess_text(title)
    batch = tokenizer(
        [title], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        num_beams=25,  # ビームサーチの探索幅
        diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        num_beam_groups=25,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)

generated_texts = generate_text_from_model(title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=10)

Writing predictor.py


In [4]:
%%writefile train_and_create_endpoint.py
# SageMaker notebookでendpointを作成
import sagemaker
from sagemaker.estimator import Estimator

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


estimator = Estimator(
    image_uri="aws_image_uri",
    role=sagemaker.get_execution_role(),
    instance_type="ml.g4dn.2xlarge",
    instance_count=1,
)
estimator.fit()
# endpoint deploy
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.g4dn.2xlarge')

result = predictor.predict(data='For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing block within the O-RU. In the UL, OFDM phase compensation (for all channels except PRACH) [3GPP TS38.211 clause 5.4], FFT, CP removal andDigital beamforming functions reside in the O.RU, while the rest of the PHY functions including resource element de-mapping, equalization, de-modulation, rate de-matching and de-coding reside in O-DU. If the same interface and split point is used for DL and UL, specification effort can be reduced.')
# delete endpoint
sagemaker.predictor.Predictor.delete_endpoint(predictor, delete_endpoint_config=True)

Writing train_and_create_endpoint.py


In [40]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::523358537305:role/service-role/AmazonSageMaker-ExecutionRole-20210813T155988
sagemaker session region: us-east-2


In [78]:
import sagemaker
from sagemaker.estimator import Estimator

estimator = Estimator(
    image_uri="523358537305.dkr.ecr.us-east-2.amazonaws.com/t5-model:latest",
    role=sagemaker.get_execution_role(),
    instance_type="ml.g4dn.2xlarge",
    instance_count=1,
)
estimator.fit()

2022-02-10 09:50:30 Starting - Starting the training job...
2022-02-10 09:50:31 Starting - Launching requested ML instancesProfilerReport-1644486629: InProgress
...
2022-02-10 09:51:27 Starting - Preparing the instances for training.........
2022-02-10 09:53:00 Downloading - Downloading input data
2022-02-10 09:53:00 Training - Downloading the training image............
2022-02-10 09:54:55 Training - Training image download completed. Training in progress..#015Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]#015Downloading: 100%|██████████| 1.58k/1.58k [00:00<00:00, 2.61MB/s]
#015Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]#015Downloading:   0%|          | 6.11M/1.63G [00:00<00:26, 61.0MB/s]#015Downloading:   1%|          | 14.3M/1.63G [00:00<00:21, 73.6MB/s]#015Downloading:   1%|▏         | 23.3M/1.63G [00:00<00:19, 81.0MB/s]#015Downloading:   2%|▏         | 32.6M/1.63G [00:00<00:18, 85.6MB/s]#015Downloading:   3%|▎         | 42.1M/1.63G [00:00<00:17, 88.9MB/s]#015

In [79]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.g4dn.2xlarge')

------!

In [80]:
result = predictor.predict(data='For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing block within the O-RU. In the UL, OFDM phase compensation (for all channels except PRACH) [3GPP TS38.211 clause 5.4], FFT, CP removal andDigital beamforming functions reside in the O.RU, while the rest of the PHY functions including resource element de-mapping, equalization, de-modulation, rate de-matching and de-coding reside in O-DU. If the same interface and split point is used for DL and UL, specification effort can be reduced.')
result

b'{"0": "Check the value of ecpriPayload.", "1": "Confirm that the value of SoltID is output normally.", "2": "Confirm the Gain. Refer to the attachment for the confirmation method.", "3": "Verify that the block floating point (1).", "4": "In \\"Table 5-1: Native Ethernet frame with VLAN in the O-RAN(CUS) specification.", "5": "Check the value of CU_Port_ID.", "6": "In \\"Table 5-1 : cpLength Use\\".", "7": "Check the value of CU_Port_ID. Refer to the attachment for the confirmation method.", "8": "Verify that the block floating point (1)._x000D_ Make sure that the relevant parameter is counted up.", "9": "Check the value of CU_Port_ID. Refer to the attachment for the confirmation method.ecpriPayload."}'

In [81]:
result.decode()

'{"0": "Check the value of ecpriPayload.", "1": "Confirm that the value of SoltID is output normally.", "2": "Confirm the Gain. Refer to the attachment for the confirmation method.", "3": "Verify that the block floating point (1).", "4": "In \\"Table 5-1: Native Ethernet frame with VLAN in the O-RAN(CUS) specification.", "5": "Check the value of CU_Port_ID.", "6": "In \\"Table 5-1 : cpLength Use\\".", "7": "Check the value of CU_Port_ID. Refer to the attachment for the confirmation method.", "8": "Verify that the block floating point (1)._x000D_ Make sure that the relevant parameter is counted up.", "9": "Check the value of CU_Port_ID. Refer to the attachment for the confirmation method.ecpriPayload."}'

In [77]:
sagemaker.predictor.Predictor.delete_endpoint(predictor, delete_endpoint_config=True)

In [86]:
import boto3

client = boto3.client('sagemaker-runtime')
# custom_attributes = "idxxxxx-yamazo-qiita-aaaa-1234567890ab" # トレース用ID
endpoint_name = "t5-model-2022-02-10-10-43-00-365"   # 推論エンドポイント名
content_type = "text/csv"                                    # コンテンツタイプ
payload = 'For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing block within the O-RU. In the UL, OFDM phase compensation (for all channels except PRACH) [3GPP TS38.211 clause 5.4], FFT, CP removal andDigital beamforming functions reside in the O.RU, while the rest of the PHY functions including resource element de-mapping, equalization, de-modulation, rate de-matching and de-coding reside in O-DU. If the same interface and split point is used for DL and UL, specification effort can be reduced.'
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    Body=payload
    )
tmp = response['Body'].read()
print(response['Body'].read())


b''


In [91]:
import json
a = json.loads(tmp.decode())
print(type(a))

<class 'dict'>


In [92]:
a["0"]

'Check the value of ecpriPayload.'

In [13]:
# wandbで可視化させるための前準備
import wandb
CONFIG = dict(
    batch_size_train = settings.batch_size_train,
    batch_size_valid = settings.batch_size_valid,
    epochs = settings.epochs
)
wandb.init(project="TestItem_T5_oranONLY_0131", entity="sasanoshouta")
wandb.config = CONFIG

In [5]:
# データ読み込み
import pandas as pd
# df = pd.read_csv('/root/myocr/oran_train_to_T5.csv', index_col=0)
df_oran = pd.read_csv('/root/myocr/oran_train_to_T5_hand_oran_1_7.csv', index_col=0)
oran_data = df_oran.rename(columns={'input': 'title', '確認事項': 'body'})
gc.collect()

0

In [26]:
df_oran.loc[10, 'input']

'2.1.2 Selected Split 7-2x (UL) 17 6.1.3.2.2 UL Gain Definition 99 6.3.2 DL/UL Data 103 6.3.3 UL/DL Data Coding of Information Elements 105 Spatial stream: the data flow on the DL associated with precoded data (may be same as layers or different if there is expansion in the precoding), and on UL associated with the number of outputs from the digital beamforming (sometimes called “beams”).  UL: UpLink : data flow away from the radiating antenna (generally on the LLS interface) The inclusion of these two O-RU categories has certain implications for the LTE and NR functional splits in both DL and UL which are reflected in the following sections.  In particular, for a Category B O-RU to implement precoding for LTE TM2-TM4 some special C-Plane instructions need to be provided to the O-RU from the O-DU; this is described in a later section of this document.  For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing blo

In [26]:
df_suiikei = pd.read_csv('/root/myocr/suiikei_pj.csv', index_col=0)
# data = df_oran.rename(columns={'input': 'title', '確認事項': 'body'})
suiikei_data = df_suiikei.drop(['input', '確認事項', '確認ポイント'], axis=1).rename(columns={'input_en': 'title', '確認事項_en': 'body'})
# suiikei_data.head()
gc.collect()

0

In [26]:
df_suiikei.loc[0, 'input_en']

'[\'Si\', \'Su\', \'Te\', \'Mu\', \'Kan\', \'Sei\', \' \', \' \', \' \', \' \', \' \', \'\\u3000\', \' \', \' \', \'3\', \'Si\', \'Su\', \'Te\', \'Mu\', \'Kan\', \'Sei\', \'\\u3000\', \'illustration\', \'1\', \'-\', \'1\', \'upon\', \'minutes\', \'detached\', \'moulds\', \'waters\', \'rank\', \'TOTAL\', \'of\', \'Si\', \'Su\', \'Te\', \'Mu\', \'Kan\', \'Sei\', \'illustration\', \'to\', \'indication\', \'Su\', \'.\', \'Ma\', \'Was\', \',\', \'illustration\', \'1\', \'-\', \'2\', \'upon\', \'minutes\', \'detached\', \'moulds\', \'waters\', \'rank\', \'TOTAL\', \'of\', \'Yoo\', \'—\', \'Su\', \'Ke\', \'—\', \'Su\', \'to\', \'indication\', \'Su\', \'.\', \'illustration\', \'1\', \'-\', \'1\', \'\\u3000\', \'Si\', \'Su\', \'Te\', \'Mu\', \'Kan\', \'Sei\', \'illustration\', \'\\u3000\', \'waters\', \'rank\', \'TOTAL\', \'parents\', \'machine\', \'is\', \',\', \'waters\', \'rank\', \'TOTAL\', \'child\', \'machine\', \'Ma\', \'Was\', \'is\', \'midst\', \'Tsugi\', \'vessel\', \'with\', \'of\', 

In [6]:
# data = pd.concat([oran_data, suiikei_data], axis=0).reset_index(drop=True)
# no_data = [i for i in range(len(data)) if data.loc[i, 'title'] == '[]']
# data = data.drop(data.index[no_data]).reset_index(drop=True)
no_data = [i for i in range(len(oran_data)) if oran_data.loc[i, 'title'] is np.nan]
print(no_data)
oran_data = oran_data.drop(oran_data.index[no_data]).reset_index(drop=True)
oran_data.tail()

[45, 46, 105]


,title,body
102,Note 7: O-RU |TE| is the requirement. O-RAN recommends O-RU internal split as shown in the table based on eCPRI and 802.1CM guidance. It is also acceptable for RAN solution vendor to use a different O-RU internal split to meet the same O-RU |TE| requirement. Note 7: O-RU |TE| is the requiremen...,"If the Quality Level and Clock Class are acceptable when the PTP Announce message is received, confirm to continue using the reference signal."
103,9.2.4.2 PTP 120 9.3.1 Allowed PTP and SyncE clock types and clock classes 123 9.7.1 PTP Time Synchronization procedure 135 PTP: Precision Time Protocol Frequency and time synchronization of O-DUs and O-RUs via Ethernet use Synchronous Ethernet and IEEE 1588-2008 Precision Time Protocol (PTP). Tr...,Make sure Clock Class is set to 6 or 7 in AnnounceMessage.
104,9.2.4.2 PTP 120 9.3.1 Allowed PTP and SyncE clock types and clock classes 123 9.7.1 PTP Time Synchronization procedure 135 PTP: Precision Time Protocol Frequency and time synchronization of O-DUs and O-RUs via Ethernet use Synchronous Ethernet and IEEE 1588-2008 Precision Time Protocol (PTP). Tr...,Verify that the reference time is January 1970 00:00:00 TAI or 31 December 1969 23:59:51.999918 UTC.
105,"9.7.2 System Frame Number Calculation from GPS Time 135 9.7.2 System Frame Number Calculation from GPS Time The general framework for System Frame Number (SFN) calculation from GPS (or GNSS) time is based on the following premises: From PTP time, the GPS seconds elapsed since GPS epoch (midnight...",Confirmed that no particular problem occurs when GPS seconds are received from PTP.
106,"Using the delay parameters specified for the corresponding profile for each O-RU in the timing domain, the uplink and downlink O-RU parameters to use for the timing domain can be determined as specified in section 2.3.3.1 Table 212. The transport delay parameters are constant for this method an...",Make sure that the frame number is calculated correctly from GPSseconds.


In [11]:
translate = boto3.client('translate')
for i in range(len(oran_data)):
    print("{}番目: ".format(i+1))
    print(" ")
    print("試験項目：")
    print(oran_data.loc[i, 'body'])
    print(" ")
    print("要約後の文章本文：")
    print(oran_data.loc[i, 'title'])
    print(" ")
    result = translate.translate_text(Text=oran_data.loc[i, 'title'], SourceLanguageCode='en', TargetLanguageCode='ja')
    print("要約後の文章本文（日本語訳）：")
    print(result["TranslatedText"])
    print("*"*100)

1番目: 
 
試験項目：
The IQ data sent by the simulator shall be received normally.
 
要約後の文章本文：
 IQ data, (both DL and UL) including user data, PRACH and control channels may be transmitted in compressed format. U-Plane: User Plane: refers to IQ sample data transferred between O-DU and O-RU Data Flow 1a: Flows of IQ Data in FFT frequency domain on DL; 1b: Fl flows of PRACH IQ data on UL. 3.3.1.1 Definition of IQ Power in dBFS 96 6.2.1 IQ Data Transfer procedure 101 A.3 -Law udCompParam and IQ data format 157 Annex D IQ Sample and Exponent Packetization for Different Bitwidths 176
 
要約後の文章本文（日本語訳）：
 ユーザーデータ、PRACH、および制御チャネルを含むIQデータ（DLとULの両方）は、圧縮形式で送信できます。U プレーン:ユーザープレーン:O-DU と O-RU の間で転送される IQ サンプルデータを指します。データフロー 1a: DL 上の FFT 周波数領域における IQ データのフロー; 1b: UL の PRACH IQ データのフロー 3.3.1.1 dBFS 96でのIQパワーの定義 6.2.1 IQ データ転送手順 101 A.3-Law udCompParam および IQ データ形式 157 附属書 D IQ異なるビット幅に対する標本パケット化と指数パケット化 176
****************************************************************************************************


# 文章要約フェイズ
参考記事：BARTによるテキスト要約【Python・Transformers】<br>
https://self-development.info/bart%e3%81%ab%e3%82%88%e3%82%8b%e3%83%86%e3%82%ad%e3%82%b9%e3%83%88%e8%a6%81%e7%b4%84%e3%80%90python%e3%83%bbtransformers%e3%80%91/

In [7]:
# BARTで、抜き出してきた文章の要約を行う関数（T5で文章生成による学習を行う際に、重要と思われる情報に短くまとめる為）
oran_data = txt_summarization_by_bart(oran_data)
oran_data
# data = txt_summarization_by_bart(oran_data)
# data

[2022-02-06 23:34:25.113 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-2a0e2554e33379262f4fea063cf1:48 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-02-06 23:34:25.151 pytorch-1-6-gpu-py3-ml-g4dn-xlarge-2a0e2554e33379262f4fea063cf1:48 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


Your max_length is set to 512, but you input_length is only 94. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


,title,body
0,"IQ data, (both DL and UL) including user data, PRACH and control channels may be transmitted in compressed format. U-Plane: User Plane: refers to IQ sample data transferred between O-DU and O-RU Data Flow 1a: Flows of IQ Data in FFT frequency domain on DL; 1b: Fl flows of PRACH IQ data on UL. 3...",The IQ data sent by the simulator shall be received normally.
1,"Multiple methods of forming the beam per the Beam Attributes are possible and are left as O-RU implementation choices. Beamforming phased array weights (Analog or Digital), Holographic Beamforming, Butler Matrices, Lenses, and other known and emerging techniques may be hybridized with “Channel-I...",Make sure you are using Digital Beamforming.
2,"Low-PHY: those portions of the PHY processing on the O-RU side of the fronthaul interface, including FFT/iFFT, digital beamforming, and PRACH extraction and filtering. In the DL, OFDM phase compensation [3GPP TS38.211 clause 5.4], FFT, CP removal and digital beamform functions reside in the O.RU...",Check the FFT size from the C-Plane of the DL.
3,"For LTE (e.g. TM9) and NR PDSCH with UE specific reference signals, the DL processing chain specified by 3GPP does not include a precoding operation. For each numerology (and PRACH/SSB), the freqOffset IE defines the relative spacing between the center of RE#0 of RB#0 to the component carrier. T...",Check Subcarrier spacing (mu)=1 (30khz) for DL.
4,3GPP: DownLink: data flow towards the radiating antenna (generally on the LLS interface) Spatial stream: the data flow on the DL associated with precoded data (may be same as layers or different if there is expansion in the precoding) UL: The number of outputs from the digital beamforming (somet...,Confirmed that the packet of Payload30Byte is received and becomes on_time.
5,3GPP: DownLink: data flow towards the radiating antenna (generally on the LLS interface) Spatial stream: the data flow on the DL associated with precoded data (may be same as layers or different if there is expansion in the precoding) UL: The number of outputs from the digital beamforming (somet...,"As shown in Figure 2-13: Uplink and Downlink Timing Parameter Relationship.\nAs shown in Figure 2-13: Uplink and Downlink Timing Parameter Relationship, confirm that the latency does not exceed T12_max."
6,"For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing block within the O-RU. In the UL, OFDM phase compensation (for all channels except PRACH) [3GPP TS38.211 clause 5.4], FFT, CP removal andDigital beamforming functio...","When using Ethernet, confirm that the packets are output as shown in Figure 3-1: Native Ethernet frame with VLAN in the O-RAN(CUS) specification."
7,"For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing block within the O-RU. In the UL, OFDM phase compensation (for all channels except PRACH) [3GPP TS38.211 clause 5.4], FFT, CP removal andDigital beamforming functio...","When using Ethernet, confirm that the packets are output as shown in Figure 3-1: Native Ethernet frame with VLAN in the O-RAN(CUS) specification."
8,3GPP: DownLink: data flow towards the radiating antenna (generally on the LLS interface) Spatial stream: the data flow on the DL associated with precoded data (may be same as layers or different if there is expansion in the precoding) UL: The number of outputs from the digital beamforming (somet...,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).
9,3GPP: DownLink: data flow towards the radiating antenna (generally on the LLS interface) Spatial stream: the data flow on the DL associated with precoded data (may be same as layers or different if there is expansion in the precoding) UL: The number of outputs from the digital beamforming (somet...,Confirm that the value of ecpriVersion is 1 (eCPRI version 1.0).


In [5]:
# T5Tokenizer定義
tokenizer = T5Tokenizer.from_pretrained(settings.MODEL_NAME, is_fast=True)
gc.collect()

4

In [6]:
def generate_batch(data):
    batch_src, batch_tgt = [], []
    for src, tgt in data:
        batch_src.append(src)
        batch_tgt.append(tgt)
    src_len_max = len(max(batch_src, key=len))
    batch_src2 = tokenizer(
            batch_src, max_length=src_len_max, truncation=True, padding="max_length", return_tensors="pt"
        )
    batch_tgt2 = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

    return batch_src2, batch_tgt2, batch_src, batch_tgt

In [7]:
# 学習データ、テストデータ、バリデーションデータに分ける(oran)
# random_state = np.random.randint(0,99)
random_state = 94
print(random_state)
X_train, X_test, y_train, y_test = train_test_split(
    oran_data['title'], oran_data['body'], test_size=0.2, random_state=random_state, shuffle=True
)

# oran_train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
# oran_valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]
train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)
# gc.collect()

94


In [10]:
a, b, c, d = generate_batch(train_data)

In [11]:
src_numpy = a['input_ids'].to('cpu').detach().numpy().copy()
print(src_numpy.shape, type(src_numpy), max(src_numpy[0]))

(85, 258338) <class 'numpy.ndarray'> 21286


In [10]:
def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI（正の相互情報量）の作成

    :param C: 共起行列
    :param verbose: 進行状況を出力するかどうか
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print('%.1f%% done' % (100*cnt/total))
    return M

In [11]:
res = ppmi(src_numpy)
gc.collect()

NameError: name 'src_numpy' is not defined

In [15]:
len(max(c, key=len))

258338

In [19]:
k = 0
for a, b in train_iter:
    print(k)
#     print(a['input_ids'].shape, a['attention_mask'].shape)
#     print(b['input_ids'].shape, b['attention_mask'].shape)
    for i in range(4):
        print(a['input_ids'][i])
    k += 1

0
tensor([28074,  7230,   344,   411,    18,  7874,    11,   411,    18,  8503,
           33,  7173,    38,   332,  2122,    41,  3035,  4907,    61,    11,
          332,  3710,    41,   413,  4907,   137,    37,  5790,  7230, 18253,
           15,     7,   163,     8,    97,    45,   116,     3,     9,   720,
         3231,     8,  1299,    49,    41,   448, 12989,   391,  5268,   552,
           34,    19,  1204,    44,     8, 11487,    41,   448, 15896,   391,
         7256,     5,    86,    46,     3, 16764,  1582,  1855,  1229,     6,
          175, 16735,   164,    59,    36,  3917,   788,    12, 14569, 16735,
           41,    23,     5,    15,     5,   276, 13529,   137,   304,   905,
           21,    48,     6,  1855,  7230,   398,    36,  1702,    38,     3,
            9,   620,    28,  4548,    11,  1364,  8120,     7,    10,   304,
          905,    21,  1855, 12338,    11,  5790,    97,     8, 11487,  4028,
            7,     3,     9,  6839,  2034,     5,   100,  1250

KeyboardInterrupt: 

In [93]:
# 学習データ、テストデータ、バリデーションデータに分ける(suiikei)
random_state = np.random.randint(0,99)
print(random_state)
X_train, X_test, y_train, y_test = train_test_split(
    data['title'], data['body'], test_size=0.2, random_state=random_state, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(suiikei_data['title'], suiikei_data['body'])]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)
gc.collect()

59


0

In [14]:
# このセル動かなかったやつ↑のコマンド実行してカーネルリスタートしたらできた
# T5モデル定義(学習時)
model = T5FineTuner()
model = model.to(settings.device)

optimizer = optim.Adam(model.parameters())

PAD_IDX = tokenizer.pad_token_id
best_loss = float('Inf')
best_model = None
counter = 1
# wandbマジックコマンド
wandb.watch(model, log_freq=1)
        
gc.collect()

456

In [30]:
for k, param in enumerate(model.parameters()):
    print('第{}層：'.format(k+1))
    print(param.shape)
    print("*"*100)

第1層：
torch.Size([32128, 512])
****************************************************************************************************
第2層：
torch.Size([384, 512])
****************************************************************************************************
第3層：
torch.Size([384, 512])
****************************************************************************************************
第4層：
torch.Size([384, 512])
****************************************************************************************************
第5層：
torch.Size([512, 384])
****************************************************************************************************
第6層：
torch.Size([32, 6])
****************************************************************************************************
第7層：
torch.Size([512])
****************************************************************************************************
第8層：
torch.Size([1024, 512])
**************************************************************************

TypeError: object of type 'generator' has no len()

In [36]:
# 追加学習時
# MODEL_NAME = sonoisa/t5-base-japanese
# model_dir_path = Path('model')
# MODEL_NAME = t5-small
# model_dir_path = Path('g_model')
# model_dir_path = Path('oran_model')
model = T5FineTuner()
model_dir_path = Path('hand_oran_model')
# model_dir_path = Path('hand_oran_model_1206')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)
tokenizer.save_pretrained(model_dir_path)
model.model.save_pretrained(model_dir_path)
gc.collect()

279

In [15]:
# これ時間かかりすぎるのでトレーニングジョブにして実行したい
# これ？https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/step-functions-data-science-sdk/model-train-evaluate-compare/step_functions_mlworkflow_scikit_learn_data_processing_and_model_evaluation_with_experiments.ipynb
for loop in range(1, settings.epochs + 1):

    start_time = time.time()

    loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

    elapsed_time = time.time() - start_time

    loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

    print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
        loop, settings.epochs, loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))

    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = copy.deepcopy(model)
        counter = 1
    else:
        if counter > settings.patience:
            break

        counter += 1
        
    gc.collect()

  0%|          | 0/11 [00:00<?, ?it/s]

[1/1000] train loss: 8.6673, valid loss: 4.9042 48s] counter: 1 **


  0%|          | 0/11 [00:00<?, ?it/s]

[2/1000] train loss: 4.2241, valid loss: 3.8589 45s] counter: 1 **


  0%|          | 0/11 [00:00<?, ?it/s]

...

[26/1000] train loss: 0.1795, valid loss: 3.1857 44s] counter: 19 


  0%|          | 0/11 [00:00<?, ?it/s]

[27/1000] train loss: 0.1825, valid loss: 2.5657 44s] counter: 20 


100%|██████████| 11/11 [01:43<00:00,  9.45s/it, loss=0.17] 


[28/1000] train loss: 0.1700, valid loss: 2.6707 44s] counter: 21 


In [13]:
# MODEL_NAME = sonoisa/t5-base-japanese
# model_dir_path = Path('model')
# MODEL_NAME = t5-small
# model_dir_path = Path('g_model')
# model_dir_path = Path('oran_model')
model_dir_path = Path('hand_oran_and_suiikei_model_0131')
# model_dir_path = Path('hand_oran_model_1206')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)
gc.collect()

1660

In [17]:
tokenizer.save_pretrained(model_dir_path)
best_model.model.save_pretrained(model_dir_path)
gc.collect()

0

In [14]:
tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)

In [16]:
# oran仕様書文章から試みる(hand-oran ver)
translate = boto3.client('translate')

valid_data_index = list()
yes_data_index = list()
no_data_index = list()
result = list()
for i in range(len(valid_data)):
    if valid_data[i][0] == '[]':
        no_data_index.append(i)
        valid_data_index.append(i)
    else:
        valid_data_index.append(i)
        yes_data_index.append(i)
#     valid_data_index.append(i)


index = 2
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=10)
# result.append(generated_texts[0])
print('□ 入力文章')
print(title)
print()
print('□ 生成本文')
for k, gen_txt in enumerate(generated_texts):
    result = translate.translate_text(Text=gen_txt, SourceLanguageCode='en', TargetLanguageCode='ja')
    print("{}番目の生成項目(英語)：{}".format(k+1, gen_txt))
    print("{}番目の生成項目(日本語)：{}".format(k+1, result['TranslatedText']))
    print()
print()
print('□ 正解試験項目(英語)')
print(body)
print('□ 正解試験項目(日本語)')
result_body = translate.translate_text(Text=body, SourceLanguageCode='en', TargetLanguageCode='ja')
print(result_body['TranslatedText'])
print('*'*100)

# for i in valid_data_index:
#     index = i
#     title = valid_data[index][0]
#     body = valid_data[index][1]
#     generated_texts = generate_text_from_model(title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=10)
#     # result.append(generated_texts[0])
#     print('□ 入力文章')
#     print(title[-100:])
#     print()
#     print('□ 生成本文')
#     for k, gen_txt in enumerate(generated_texts):
#         result = translate.translate_text(Text=gen_txt, SourceLanguageCode='en', TargetLanguageCode='ja')
#         print("{}番目の生成項目(英語)：{}".format(k+1, gen_txt))
#         print("{}番目の生成項目(日本語)：{}".format(k+1, result['TranslatedText']))
#         print()
#     print()
#     print('□ 正解試験項目(英語)')
#     print(body)
#     print('□ 正解試験項目(日本語)')
#     result_body = translate.translate_text(Text=body, SourceLanguageCode='en', TargetLanguageCode='ja')
#     print(result_body['TranslatedText'])
#     print('*'*100)

# for i in valid_data_index:
#     index = i
#     title = valid_data[index][0]
#     body = valid_data[index][1]
#     generated_texts = generate_text_from_model(
#         title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=10
#     )
#     result.append(generated_texts[0])
#     print('□ タイトル')
#     print(title[-10:])
#     print()
#     print('□ 生成本文')
#     print(generated_texts[0])
#     print()
#     print('□ 教師データ本文')
#     print(body)
#     print('*'*100)

□ 入力文章
For LTE TM5-10 and NR, no special instructions are needed because the precoding may be included in a digital beamforming processing block within the O-RU. In the UL, OFDM phase compensation (for all channels except PRACH) [3GPP TS38.211 clause 5.4], FFT, CP removal andDigital beamforming functions reside in the O.RU, while the rest of the PHY functions including resource element de-mapping, equalization, de-modulation, rate de-matching and de-coding reside in O-DU. If the same interface and split point is used for DL and UL, specification effort can be reduced.

□ 生成本文
1番目の生成項目(英語)：Confirm that the value of ecpriPayload is 1 (eCPRI version 1.0).
1番目の生成項目(日本語)：ecPriPayLoad の値が 1 (ecPRI バージョン 1.0) であることを確認します。

2番目の生成項目(英語)：Check the value of ecpriPayload.
2番目の生成項目(日本語)：ecpriPayLoad の値を確認します。

3番目の生成項目(英語)：When using Ethernet, confirm that the frameID output is output correctly.
3番目の生成項目(日本語)：イーサネットを使用する場合は、FrameID 出力が正しく出力されることを確認します。

4番目の生成項目(英語)：Make sure you are using Etherne

In [29]:
# !pip install openpyxl

In [24]:
# 推論結果と正解データの内容比較

df_y = pd.DataFrame(y_test)
valid_index = list(df_y.index)

sample = pd.read_excel('/root/myocr/oran_test_content.xlsx', engine='openpyxl', header=1).reset_index(drop=True)\
                        .drop(['Unnamed: 0', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 14'], axis=1)\
                        .drop(['大項目', '中項目', '小項目1', '小項目2', '小項目3', '小項目4', 'Reference', '項番'], axis=1)\
                        .dropna(how='any').reset_index(drop=True)

df_base = pd.read_csv('/root/myocr/suiikei_pj.csv', index_col=0)
df_suiikei = df_base.drop(['input', '確認ポイント', 'input_en', '確認事項_en'], axis=1)
# data = df_oran.rename(columns={'input': 'title', '確認事項': 'body'})
# suiikei_data = df_suiikei.drop(['input', '確認事項', '確認ポイント'], axis=1).rename(columns={'input_en': 'title', '確認事項_en': 'body'})
df_both = pd.concat([sample, df_suiikei], axis=0).reset_index(drop=True)
valid_grandtruth = df_both.iloc[valid_index]
tmp = pd.concat([valid_grandtruth, df_y], axis=1).rename(columns={'body': '確認事項_英語'})
df_result = pd.DataFrame(result, index=valid_index).rename(columns={0: '推論結果_英語'})
df_all = pd.concat([tmp, df_result], axis=1)
df_all['推論結果_日本語'] = '[]'

translate = boto3.client('translate')
index = list(df_all.index)
print(df_result.columns)
print(df_all.columns)

for ind in index:
    result = translate.translate_text(Text=df_all.loc[ind, '推論結果_英語'], SourceLanguageCode='en', TargetLanguageCode='ja')
    df_all.loc[ind, '推論結果_日本語'] = result['TranslatedText']
    
df_all = df_all.reset_index(drop=True)
gc.collect()

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


Index(['推論結果_英語'], dtype='object')
Index(['確認事項', '確認事項_英語', '推論結果_英語', '推論結果_日本語'], dtype='object')


1479

In [25]:
df_all['cos_nearly'] = 0
df_all.head()

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語,cos_nearly
0,ULの場合は0。DLの場合は1であることを確認する。,Check that it is 1.,The received data matches the content sent from the cloud,受信したデータは、クラウドから送信されたコンテンツと一致します。,0
1,bfwIqWidthを確認する。bfwCompMeth=1(block floating point)であることを確認する。,Confirm that the output is from bfwQ_0~31.,The average value obtained will remain in the log,取得した平均値はログに残ります。,0
2,BandSector_IDの値を確認する。,Check the value of CU_Port_ID.,The sender station code is set,送信側局コードが設定されている,0
3,観測モード時に[0010 0000]が設定されていること,[0000 0100] is set at the time of water level gauge communication alarm,The sender station code of the terminal (slave unit) that made the transmission is set.,送信を行った端末（スレーブユニット）の送信側局コードが設定されます。,0
4,PC接続により、局番設定を受信した場合、グループIDにて通知された値を端末のグループIDとして設定すること。,"When the water level hysteresis setting is received by PC connection, the value notified in the water level hysteresis should be set as the water level hysteresis.",The sender station code of the terminal (slave unit) that made the transmission is set.,送信を行った端末（スレーブユニット）の送信側局コードが設定されます。,0


In [26]:
# 正解試験項目と推論試験項目の類似度計算
for i in range(len(df_all)):
    tmp = cos_sim(vectorize(df_all.loc[i, '確認事項_英語']), vectorize(df_all.loc[i, '推論結果_英語']))
    df_all.loc[i, 'cos_nearly'] = tmp
#     print('正解文章: ', df_all.loc[i, '確認事項_英語'])
#     print('生成文章: ', df_all.loc[i, '推論結果_英語'])
#     print('コサイン類似度: ', tmp)
#     print('*'*100)
    
gc.collect()

Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from

****************************************************************************************************


Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from

****************************************************************************************************


Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from

...

Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from

****************************************************************************************************


Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at hand_oran_and_suiikei_model_0107 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from

****************************************************************************************************


1415

In [30]:
df_all.to_csv('/root/myocr/result_22_1_11.csv')

In [30]:
df_all.head()

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語,cos_nearly
0,ULの場合は0。DLの場合は1であることを確認する。,Check that it is 1.,"When the station number setting is received through PC connection, set the value notified by an uplink data transmission command.",PC接続で局番号設定を受信した場合は、上りリンクデータ送信コマンドで通知される値を設定する。,0.900171
1,bfwIqWidthを確認する。bfwCompMeth=1(block floating point)であることを確認する。,Confirm that the output is from bfwQ_0~31.,"If there is no problem with the water level threshold setting instruction, set [0001 0110] to the instruction type",水位閾値設定指示に問題がなければ、指示種別に [0001 0110] を設定してください。,0.888409
2,BandSector_IDの値を確認する。,Check the value of CU_Port_ID.,"When the station number setting is received through PC connection, set the value notified by an uplink data transmission command.",PC接続で局番号設定を受信した場合は、上りリンクデータ送信コマンドで通知される値を設定する。,0.905935
3,観測モード時に[0010 0000]が設定されていること,[0000 0100] is set at the time of water level gauge communication alarm,"When the station number setting is received through PC connection, set the value notified by an uplink data transmission command.",PC接続で局番号設定を受信した場合は、上りリンクデータ送信コマンドで通知される値を設定する。,0.926812
4,PC接続により、局番設定を受信した場合、グループIDにて通知された値を端末のグループIDとして設定すること。,"When the water level hysteresis setting is received by PC connection, the value notified in the water level hysteresis should be set as the water level hysteresis.","When the station number setting is received through PC connection, set the value notified by an uplink data transmission command.",PC接続で局番号設定を受信した場合は、上りリンクデータ送信コマンドで通知される値を設定する。,0.994383


In [27]:
# コサイン類似度o.99以上の結果
print(len(df_all[df_all['cos_nearly'] > 0.99]) / len(df_all))

0.30526315789473685


### コサイン類似度の閾値0.99以上だと仮定すると、正解率約30%に相当

In [28]:
df_all[df_all['cos_nearly'] > 0.99]

,確認事項,確認事項_英語,推論結果_英語,推論結果_日本語,cos_nearly
8,上位から要求された場合、最新の状況を報告すること,"When the slave unit is set to the monitoring mode (no monitoring data is sent to the parent unit) with the mode setting instruction, the monitoring mode (no monitoring data is sent to the parent unit) with the instruction code [0000 0000], the station number of the target slave unit, and the mod...","When the monitor mode (with monitoring data sent to the parent unit) [0000 0011], select one of those who want an instruction in which it is instructed.",監視モード（親機に監視データを送信した状態）[00000011] の場合は、指示された命令を希望する人のいずれかを選択します。,0.998294
11,クラウド側で受信した内容が送信の内容と一致すること,The received data matches the content sent from the cloud,The received data matches the content sent from the cloud,受信したデータは、クラウドから送信されたコンテンツと一致します。,1.000000
12,範囲外の値が指定されていた場合、受信応答でエラーを返すこと。,"When the parameter setting instruction is specified in the instruction code, the parameter type is [0000 0010], and the parameter 2 is other than [0], the standard deviation factor (X) is specified.",When the parameter setting instruction is specified in the instruction code and it should be referred to as an error.,命令コードにパラメータ設定指令が指定されていて、エラーと呼ぶべき場合。,0.998452
13,クラウド側で受信した内容が送信の内容と一致すること,The received data matches the content sent from the cloud,The received data matches the content sent from the cloud,受信したデータは、クラウドから送信されたコンテンツと一致します。,1.000000
14,RS-485 PC通信終了指示を受けた場合、PCとの通史を停止し、水位計との通信を再確立すること,The new log item for the water level gauge should be stored in the external FLASH.,The new log item for the water level gauge should be stored in the external FLASH.,水位計の新しいログアイテムは、外部フラッシュに保存する必要があります。,1.000000
18,IoTアクセス端末のコンソールツールで接続可能であること,API connection on the terminal side of the NESIC version of Private Lora can be made via PC connection,API connection on the terminal side of the NESIC version can be made via PC connection,NESIC版の端末側でのAPI接続はPC接続で可能,0.997176
20,シミュレータが送信したIQデータを正常に受信すること。,The IQ data sent by the simulator shall be received normally.,The measured time is set.,計測した時間が設定されます。,0.992772
21,タイムアウトエラーがログに残っていること,The water level measurement error remains in the log,The sender station code of the terminal (slave unit) that made the transmission is set.,送信を行った端末（スレーブユニット）の送信側局コードが設定されます。,0.990818
23,受信したデータの命令コードが[0000 0010]であった場合、送信元局番を取り出すこと,Retrieve the sender station code when the instruction code of the received data is [0000 0011],Retrieve the first log number when the instruction code of receiving an instruction to report log data is [0000 0011],ログデータを報告する命令を受信する命令コードが [0000 0011] の場合、最初のログ番号を取得する,0.998282
26,上位から送信要求受付レスポンスを受けた場合、上りデータ送信コマンドで32byteのデータ送信を行う。送信データは送信の度にインクリメントされること。,To collect data on the reception level of the beacon signal (only the first subframe) in subframe units and transmit it to the PC,To collect data on the reception level of no received signal (only a subframe) in subframe units and transmit it to the PC as collected data for consumption purposes,受信信号がない（サブフレームのみ）の受信レベルのデータをサブフレーム単位で収集し、収集したデータとしてPCに送信して消費する,0.997617


In [44]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
model = T5Model.from_pretrained(model_dir_path)

input_ids = tokenizer("Studies have been shown that owning a dog is good for you", return_tensors="pt").input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies have been shown that owning a dog is good for you", return_tensors="pt").input_ids  # Batch size 1

# forward pass
outputs = model(input_ids, decoder_input_ids=decoder_input_ids)
last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at hand_oran_and_suiikei_model_1224 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
